In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose([transforms.ToTensor()])
train_set = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)

In [3]:
test_set = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=False)

In [4]:
print(train_set.data.shape)
print(test_set.data.shape)

torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])


In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()

        self.conv1 = nn.Conv2d(1,10,5)
        self.conv2 = nn.Conv2d(10,20,5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)
    
    def forward(self,x):
        x = F.relu((F.max_pool2d(self.conv1(x),2)))
        x = F.relu((F.max_pool2d(self.conv2_drop(self.conv2(x)),2)))
        x = x.view(-1,320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x,training=self.training)
        x = self.fc2(x)

        return F.softmax(x)
    

In [6]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [7]:
def train(epoch):
    model.train()
    for batch_idx, (data,target) in enumerate(test_loader):
        data,target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output,target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)} / {len(train_loader.dataset)} ({100 * batch_idx / len(train_loader):0f}%)]\t{loss.item():.6f}")
            


In [8]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data,target in test_loader:
            data,target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output,target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print(f"\nTest set: Average loss: {test_loss: 0.4f}, Accuracy {correct}/{len(test_loader.dataset)}  ({100 * correct / len(test_loader.dataset):.0f}%\n)")



In [9]:
for epoch in range(1, 10):
    train(epoch)
    test()

/tmp/ipykernel_1649133/2312631035.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch: 1 [0 / 60000 (0.000000%)]	2.302935
Train Epoch: 1 [2000 / 60000 (3.333333%)]	2.286932
Train Epoch: 1 [4000 / 60000 (6.666667%)]	2.041288
Train Epoch: 1 [6000 / 60000 (10.000000%)]	2.016140
Train Epoch: 1 [8000 / 60000 (13.333333%)]	1.882420

Test set: Average loss:  0.0172, Accuracy 7533/10000  (75%

Train Epoch: 2 [0 / 60000 (0.000000%)]	1.778316
Train Epoch: 2 [2000 / 60000 (3.333333%)]	1.850412
Train Epoch: 2 [4000 / 60000 (6.666667%)]	1.757574
Train Epoch: 2 [6000 / 60000 (10.000000%)]	1.770336
Train Epoch: 2 [8000 / 60000 (13.333333%)]	1.696649

Test set: Average loss:  0.0161, Accuracy 8690/10000  (87%

Train Epoch: 3 [0 / 60000 (0.000000%)]	1.692958
Train Epoch: 3 [2000 / 60000 (3.333333%)]	1.719262
Train Epoch: 3 [4000 / 60000 (6.666667%)]	1.717839
Train Epoch: 3 [6000 / 60000 (10.000000%)]	1.717695
Train Epoch: 3 [8000 / 60000 (13.333333%)]	1.608770

Test set: Average loss:  0.0158, Accuracy 8947/10000  (89%

Train Epoch: 4 [0 / 60000 (0.000000%)]	1.655985
Train E